## 从雅虎财经网站获取股票数据

In [13]:
import urllib
import os
import datetime
import pandas as pd
import numpy as np

In [2]:
# 下载整个股票数据
def retrive_stock_data(stockid, folder):
    print('downloading %s to %s' % (stockid, folder))
    url = 'http://table.finance.yahoo.com/table.csv?s=%s' % (stockid)
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.isdir(folder):
        os.mkdir(folder)
    urllib.urlretrieve(url, fname)

In [24]:
retrive_stock_data('600690.ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


In [3]:
# 更新股票数据，如果不存在，则下载。如果存在，则只更新最近日期的数据
def update_stock_data(stockid, folder):
    fname = os.path.join(folder, '%s.csv' % stockid.split('.')[0])
    if not os.path.exists(fname):
        retrive_stock_data(stockid, folder)
        return
    
    data = pd.read_csv(fname, index_col='Date', parse_dates=True)
    
    last_date = data.iloc[0:1].index.tolist()[0]
    today = pd.Timestamp(datetime.date.today())
    if today - last_date < pd.Timedelta(days=2):
        print('Nothing to update. %s last date is %s.' % (stockid, last_date))
        return
    
    print('updatting %s to from %s to %s' % (stockid, last_date.date(), today.date()))
    query = [
        ('a', last_date.month - 1),
        ('b', last_date.day),
        ('c', last_date.year),
        ('d', today.month - 1),
        ('e', today.day),
        ('f', today.year),
        ('s', stockid),
    ]
    url = 'http://table.finance.yahoo.com/table.csv?%s' % urllib.urlencode(query)
    temp_file = fname + '.tmp'
    urllib.urlretrieve(url, temp_file)
    update_data = pd.read_csv(temp_file, index_col='Date', parse_dates=True)
    data = data.append(update_data)
    data.sort_index(ascending=False, inplace=True)
    data.to_csv(fname, mode='w')
    os.unlink(temp_file)

In [4]:
update_stock_data('600690.ss', 'yahoo-data')

downloading 600690.ss to yahoo-data


IOError: [Errno socket error] [Errno 10061] No connection could be made because the target machine actively refused it

### 获取所有的股票列表

In [21]:
sh = pd.read_csv('SH.txt', header=None, names=['name', 'id'], dtype={'id': np.string0})
sh['postfix']='.ss'
sh

,name,id,postfix
0,浦发银行,600000,.ss
1,邯郸钢铁,600001,.ss
2,齐鲁石化,600002,.ss
3,ST东北高,600003,.ss
4,白云机场,600004,.ss
5,武钢股份,600005,.ss
6,东风汽车,600006,.ss
7,中国国贸,600007,.ss
8,首创股份,600008,.ss
9,上海机场,600009,.ss


In [20]:
sz = pd.read_csv('SZ.txt', header=None, names=['name', 'id'], dtype={'id': np.string0})
sz['postfix']='.sz'
sz

,name,id,postfix
0,平安银行,000001,.sz
1,万科A,000002,.sz
2,PT金田A,000003,.sz
3,国农科技,000004,.sz
4,世纪星源,000005,.sz
5,深振业A,000006,.sz
6,零七股份,000007,.sz
7,神州高铁,000008,.sz
8,中国宝安,000009,.sz
9,深华新,000010,.sz


In [32]:
slist = pd.concat([sh, sz])

In [33]:
for i in range(len(slist)):
    s = slist.iloc[i]
    print s['id'] + s['postfix']

 600000.ss
 600001.ss
 600002.ss
 600003.ss
 600004.ss
 600005.ss
 600006.ss
 600007.ss
 600008.ss
 600009.ss
 600010.ss
 600011.ss
 600012.ss
 600015.ss
 600016.ss
 600017.ss
 600018.ss
 600019.ss
 600020.ss
 600021.ss
 600022.ss
 600023.ss
 600026.ss
 600027.ss
 600028.ss
 600029.ss
 600030.ss
 600031.ss
 600033.ss
 600035.ss
 600036.ss
 600037.ss
 600038.ss
 600039.ss
 600048.ss
 600050.ss
 600051.ss
 600052.ss
 600053.ss
 600054.ss
 600055.ss
 600056.ss
 600057.ss
 600058.ss
 600059.ss
 600060.ss
 600061.ss
 600062.ss
 600063.ss
 600064.ss
 600065.ss
 600066.ss
 600067.ss
 600068.ss
 600069.ss
 600070.ss
 600071.ss
 600072.ss
 600073.ss
 600074.ss
 600075.ss
 600076.ss
 600077.ss
 600078.ss
 600079.ss
 600080.ss
 600081.ss
 600082.ss
 600083.ss
 600084.ss
 600085.ss
 600086.ss
 600087.ss
 600088.ss
 600089.ss
 600090.ss
 600091.ss
 600092.ss
 600093.ss
 600094.ss
 600095.ss
 600096.ss
 600097.ss
 600098.ss
 600099.ss
 600100.ss
 600101.ss
 600102.ss
 600103.ss
 600104.ss
 600105.ss